# Real-time Detecting in Colab

In [5]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import time
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet import preprocess_input
from keras.models import load_model
import IPython.display as display
from google.colab.output import eval_js
from base64 import b64decode
import PIL.Image
import io

model = load_model('"C:\Users\dell\Downloads\MobileNetFaceSpoof (4).h5"', custom_objects={})
emotion_labels_2 = ["live", "spoof"]

# JavaScript function to capture image from webcam
def take_photo(filename='photo.jpg', quality=0.8):
    js = f"""
    async function takePhoto() {{
        const div = document.createElement('div');
        document.body.appendChild(div);
        const video = document.createElement('video');
        video.style.display = 'block';
        document.body.appendChild(video);

        const stream = await navigator.mediaDevices.getUserMedia({{video: true}});
        video.srcObject = stream;
        await new Promise((resolve) => (video.onloadedmetadata = resolve));
        video.play();

        await new Promise((resolve) => setTimeout(resolve, 1000));

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);

        stream.getTracks().forEach(track => track.stop());
        video.remove();

        const dataURL = canvas.toDataURL('image/jpeg', {quality});
        return dataURL;
    }}
    takePhoto();
    """
    data = eval_js(js)
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

image_path = take_photo()

frame = cv2.imread(image_path)

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

for (x, y, w, h) in faces:
    face_roi = frame[y:y+h, x:x+w]
    face_roi = cv2.resize(face_roi, (224, 224), interpolation=cv2.INTER_AREA)
    face_rgb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
    face_array = img_to_array(face_rgb)
    face_array = np.expand_dims(face_array, axis=0)
    face_array = preprocess_input(face_array)

    prediction = model.predict(face_array)[0]
    if prediction < 0.7:
        label = 'Real'
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    else:
        label = 'Spoof'
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,0.9, (0, 0, 255), 2)

    #label = emotion_labels_2[emotion_index]
    print(prediction,label)

cv2_imshow(frame)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 3-4: truncated \UXXXXXXXX escape (<ipython-input-5-0df75e32e121>, line 14)

# Detecting From Video

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet import preprocess_input
from keras.models import load_model
from google.colab.patches import cv2_imshow

model = load_model('/content/drive/MyDrive/Face_anti_spoof/MobileNetFaceSpoof.h5', custom_objects={})
emotion_labels_2 = ["live", "spoof"]

video_path = 'WIN_20250117_17_48_20_Pro.mp4'

cap = cv2.VideoCapture(video_path)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (224, 224), interpolation=cv2.INTER_AREA)
        face_rgb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        face_array = img_to_array(face_rgb)
        face_array = np.expand_dims(face_array, axis=0)
        face_array = preprocess_input(face_array)

        prediction = model.predict(face_array)[0]
        if prediction < 0.7:
            label = 'Real'
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        else:
            label = 'Spoof'
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,0.9, (0, 0, 255), 2)

    #label = emotion_labels_2[emotion_index]
    print(prediction,label)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

from IPython.display import display, HTML
import base64

def show_video(video_path):
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + base64.b64encode(mp4).decode()
    display(HTML(f'<video width="640" height="480" controls><source src="{data_url}" type="video/mp4"></video>'))

show_video('output.mp4')


Output hidden; open in https://colab.research.google.com to view.

# Detecting Real-time From Local jupyter Notebook

In [ ]:
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet import preprocess_input
from keras.models import load_model

model = load_model('MobileNetFaceSpoof.h5', custom_objects={})
emotion_labels_2 = ["live", "spoof"]

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (224, 224), interpolation=cv2.INTER_AREA)
        face_rgb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        face_array = img_to_array(face_rgb)
        face_array = np.expand_dims(face_array, axis=0)
        face_array = preprocess_input(face_array)

        prediction = model.predict(face_array)[0]
        if prediction < 0.7:
            label = 'Real'
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        else:
            label = 'Spoof'
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,0.9, (0, 0, 255), 2)

    cv2.imshow('Real-Time Face Spoof Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

